# Goals

Implement the simple generating process given in "Algorithm 2" in our simpliciality file on Overleaf, with weighting scheme given by Example 5.1 and $\Theta_{k}$ being exactly the set of all previously-seen hyperedges (optionally allowing "the full graph" with some multiplicity)

In [1]:
import numpy as np

In [88]:
# Create a graph according to Algorithm 2 in our Overleaf.

# V: integer - number of vertices.
# n: nd array of integers of length K_max, with n[k] being the number of hyperedges of size k+2. 
# p: ndarray of height V and width K_max, with p[v,k] being the weight of vertex v for hypergraphs of size k. If an array of length V is passed, it should be copied K_max times to make an array.
# beta: ndarray of height V and width K_max, with beta[v,k] being the weighting scheme seen in Example 5.1. If an array of length V or K_max is passed, it should be copied to make the 2d array. If a number is passed, it should be copied to make the array.
# Global (default = None): if not None, should be a list of integers of length K_max, with Global[k] saying how often to add the set "all vertices" to $\Theta_{k}$. If an integer is passed, it should be copied K_max times to make a list.

# NOTE: I make a token effort to make this modular by allowing one to sub in other strategies for e.g. "simple_upweight" (which would require replacing "get_alpha" with a generic "get_parameters" and replacing all of the preprocessing stuff). 

def Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10):
    # Preprocessing to make sure everything is the right shape.
    K_max = n.shape[0]
    p = Proc_p(p,K_max)
    beta = Proc_beta(beta,V,K_max)

    # Initialize
    H_list = [] # Elements will be the hyperedges (set objects)
    
    for k in range(K_max-1,-1,-1): # loop over size of hyperedge, from biggest to smallest
        # Freeze the hyperedge list, as the algorithm does
        Theta = H_list.copy()
        if k == (K_max-1): # In this case the list is empty, so add the "global" background. Note I'm wasting computational time doing this, since I don't need to compute alpha here. Oh well.
            Theta.append(np.array([x for x in range(V)]))
        if Global is not None:
            for i in range(Global[k]): # OK, this is surely a silly way to do this. Oh well.
                Theta.append(np.array([x for x in range(V)]))
        alpha = get_alpha(Theta,beta,V,k,eps=eps) # Compute alpha.
        for ind in range(n[k]): # make n[k] hyperedges
            adj_p = simple_upweight(Theta,p,V,alpha,beta) # Choose an element of theta, adjust p.
            H_list.append(make_CL_edge(k+2,adj_p))
    return(H_list)
    

In [44]:
# Makes a single hyperedge of size k according to soft Chung-Lu with weights p.
# CHECKED BASIC FUNCTION
def make_CL_edge(k,p):
    return(np.random.choice(V,size=k,p=p))

In [57]:
# Computes alpha, as in Example 5.1 of Overleaf. Note that at this point k is "fixed" and so is not included as an index where it is not needed.
# CHECKED BASIC FUNCTION
def get_alpha(Theta,beta,V,k,eps = (0.1)**10):
    # Compute gamma
    gamma = np.zeros(V)
    for edge in Theta:
        for v in np.unique(edge): # Computing probabilities, so we don't want to deal with self-loops.
            gamma[v] = gamma[v] + 1 # Doing raw counts.
    for v in range(V):
        gamma[v] = gamma[v]/len(Theta) # normalize
        
    # Compute alpha
    alpha = np.zeros(V)
    for v in range(V):
        alpha[v] = (1 - beta[v,k]*(1-gamma[v]))/(eps + gamma[v]) # Include small "eps" value to reduce numerical issues at cost of small bias.
    return(alpha)

In [68]:
# Does the simple upweight 
# CHECKED BASIC FUNCTION
def simple_upweight(Theta,p,V,alpha,beta):
    m = len(Theta)

    S = Theta[np.random.randint(m)] # Choose a random hyperedge to "be inside"
    adj_p = np.zeros(V)
    for v in range(V):
        if v in S:
            adj_p[v] = p[v,k]*alpha[v]
        else:
            adj_p[v] = p[v,k]*beta[v,k]
    tot = sum(adj_p)
    for v in range(V):
        adj_p[v] = adj_p[v]/tot
    return(adj_p)

In [28]:
# Preprocesses p into a matrix of the appropriate size/shape.
# CHECKED BASIC FUNCTION
def Proc_p(p,K_max):
    if len(p.shape) == 2:
        return(p)
    if len(p.shape) == 1:
        V = p.shape[0]
        p = np.tile(p, K_max).reshape(K_max,V).transpose()
        return(p)
    return(-1) # error
        

In [55]:
# Preprocesses beta into a matrix of the appropriate size/shape.
# CHECKED BASIC FUNCTION
def Proc_beta(beta,V,K_max):
    res = np.zeros((V,K_max))
    if type(beta) is float:
        res.fill(beta)
        return(res)
    if len(beta.shape) == 2:
        return(beta)
    if len(beta.shape) == 1:
        if beta.shape[0] == V:
            beta = np.tile(beta, K_max).reshape(K_max,V).transpose()
            return(beta)
        if beta.shape[0] == K_max:
            beta = np.tile(beta, V).reshape(V,K_max)
            return(beta)
    return(-1) # error

## Sanity Check - does something

Do the above main function and helper functions do something plausible?

In [91]:

# Setting up choices
# p
p = np.array([0.3,2,0.45,3,8,0.22,0.5,1.2,2.3,2.5])
p = p/sum(p)
p = Proc_p(p,K_max)

#V
V = p.shape[0]

# K_max
K_max =4
k=3

# Theta
Theta = []
Theta.append([5,6,7,8])
Theta.append([1,6,3,8])
Theta.append([2,6,4,3])
Theta.append([0,1,2,3,4,5,6,7,8,9])

# beta
beta = Proc_beta(0.2,V,K_max)

# alpha
alpha = get_alpha(Theta,beta,V,k)

# n
n = np.array([0,3,3,1])

Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)

[array([4, 4, 1, 7, 7]),
 array([4, 4, 3, 1]),
 array([7, 4, 4, 4]),
 array([1, 4, 4, 4]),
 array([7, 4, 4]),
 array([4, 1, 4]),
 array([1, 4, 4])]

## Sanity check - not "too slow"

Let's see how long this takes to make a graph with a a few hundred hyperedges.



In [99]:
%%time
V = 1000
n=np.array([222,84,33,12,4,1])
p = np.asfarray([np.random.randint(15,30) for x in range(V)])
tot = sum(p)
for v in range(V):
    p[v] = p[v]/tot
beta = 0.2



CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


In [101]:
%%time
res = Soft_CL(V,n,p,beta,Global=None,eps = (0.1)**10)

CPU times: user 1.73 s, sys: 670 µs, total: 1.73 s
Wall time: 2.17 s


In [102]:
len(res)

356